In [1]:
from car_racing import CarRacing

import scipy.misc
import imageio
import numpy as np
import gym

import msvcrt
import win32api
import win32con

from multiprocess import Process, Queue, Manager
import os

from random import choice, random, randint

env = CarRacing()

D:\Users\Kristopher_2\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
bs = 32

# Device configuration
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [16]:
#Neural Network Parameters
num_epochs = 30
batch_size = bs
learning_rate = 0.001

input_size = 64 #Dimensions of the input "concatenatedTensor"
hidden_size = [32,16,8]
num_classes = 3 #Three state outputs. Acceleration. Braking. Turning Angle

#Define neural network for this part
class AdditivePolicyNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AdditivePolicyNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1]) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2]) 
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size[2], num_classes) 
        self.relu = nn.ReLU()
        
        self.fc5 = nn.Linear(8+3, 12) 
        self.relu = nn.ReLU()
        self.fc6 = nn.Linear(12, 12)
        self.relu = nn.ReLU()
        self.fc7 = nn.Linear(12, 8)
        self.relu = nn.ReLU()
        self.fc8 = nn.Linear(hidden_size[2], num_classes) 
        
        
    
    def forward(self, x):
        out = self.fc1(x.reshape(-1,input_size))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        
        
        fc4out = self.fc4(out)
        fc4out = self.relu(fc4out)
        
        
        out = self.fc5(torch.cat((out, fc4out), 1))
        out = self.relu(out)
        out = self.fc6(out)
        out = self.relu(out)
        out = self.fc7(out)
        out = self.relu(out)
        out = self.fc8(out)
        return out

model3 = AdditivePolicyNet(input_size, hidden_size, num_classes).to(device)
            
model3.load_state_dict(torch.load('CurrentConditions.torch', map_location=device), strict=False)


_IncompatibleKeys(missing_keys=['fc5.weight', 'fc5.bias', 'fc6.weight', 'fc6.bias', 'fc7.weight', 'fc7.bias', 'fc8.weight', 'fc8.bias'], unexpected_keys=[])

In [4]:
#Neural Network Parameters
num_epochs = 30
batch_size = bs
learning_rate = 0.001

input_size = 64 #Dimensions of the input "concatenatedTensor"
hidden_size = [32,16,8]
num_classes = 3 #Three state outputs. Acceleration. Braking. Turning Angle

#Define neural network for this part
class StateDetectionNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(StateDetectionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1]) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2]) 
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size[2], num_classes)  
    
    def forward(self, x):
        out = self.fc1(x.reshape(-1,input_size))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

model2 = StateDetectionNet(input_size, hidden_size, num_classes).to(device)
model2.load_state_dict(torch.load('CurrentConditions.torch', map_location=device))

<All keys matched successfully>

In [5]:
class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        esp = esp.to(device)
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar
    
image_channels = 3
model = VAE(image_channels=image_channels).to(device)
model.load_state_dict(torch.load('vae.torch', map_location=device))
model=model.to(device)

In [6]:
def unnormalize(action_array):
    mean_col1 = -0.026595786253976014
    mean_col2 = 0.21783560955090628
    mean_col3 = 0.007642476004356518
    sd_col1 = 0.10711904839787884
    sd_col2 = 0.09744389996469334
    sd_col3 = 0.043793150114235936
    col1 = (action_array[:,0]*sd_col1)+mean_col1
    col2 = (action_array[:,1]*sd_col2)+mean_col2
    col3 = (action_array[:,2]*sd_col3)+mean_col3
    return np.column_stack((col1,col2,col3))

In [7]:
#thanks to https://stackoverflow.com/a/61495130 for this code
import matplotlib
from matplotlib import pyplot as plt

class LiveLine:
    def __init__(self, graph, fmt=''):
        # LiveGraph object
        self.graph = graph
        # instant line
        self.line, = self.graph.ax.plot([], [], fmt)
        # holder of new lines
        self.lines = []

    def update(self, x_data, y_data):
        # update the instant line
        self.line.set_data(x_data, y_data)
        self.graph.update_graph()

    def addtive_plot(self, x_data, y_data, fmt=''):
        # add new line in the same figure
        line, = self.graph.ax.plot(x_data, y_data, fmt)
        # store line in lines holder
        self.lines.append(line)
        # update figure
        self.graph.update_graph()
        # return line index
        return self.lines.index(line)

    def update_indexed_line(self, index, x_data, y_data):
        # use index to update that line
        self.lines[index].set_data(x_data, y_data)
        self.graph.update_graph()


class LiveGraph:
    def __init__(self, backend='nbAgg', figure_arg={}, window_title=None, 
                 suptitle_arg={'t':None}, ax_label={'x':'', 'y':''}, ax_title=None):

        # save current backend for later restore
        self.origin_backend = matplotlib.get_backend()

        # check if current backend meets target backend
        if self.origin_backend != backend:
            print("original backend:", self.origin_backend)
            # matplotlib.use('nbAgg',warn=False, force=True)
            plt.switch_backend(backend)
            print("switch to backend:", matplotlib.get_backend())

        # set figure
        self.figure = plt.figure(**figure_arg)
        self.figure.canvas.set_window_title(window_title)
        self.figure.suptitle(**suptitle_arg)

        # set axis
        self.ax = self.figure.add_subplot(111)
        self.ax.set_xlabel(ax_label['x'])
        self.ax.set_ylabel(ax_label['y'])
        self.ax.set_title(ax_title)

        # holder of lines
        self.lines = []

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def close(self):
        # check if current beckend meets original backend, if not, restore it
        if matplotlib.get_backend() != self.origin_backend:
            # matplotlib.use(self.origin_backend,warn=False, force=True)
            plt.switch_backend(self.origin_backend)
            print("restore to backend:", matplotlib.get_backend())

    def add_line(self, fmt=''):
        line = LiveLine(graph=self, fmt=fmt)
        self.lines.append(line)
        return line

    def update_graph(self):
        self.figure.gca().relim()
        self.figure.gca().autoscale_view()
        self.figure.canvas.draw()


In [85]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image

def runSimulator(time, render=True, graph=False, graphEveryX=1):
    obs = env.reset()
    #env.render()



    t = 0
    r = 0
    scores = []
    max_reward = 0

    #while(True):
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    if render:
         env.render()
    
    PreviousImage = obs
    CurrentImage = obs
    
    if graph:
        g = LiveGraph()
        line1 = g.add_line()
        list1 = []
    
        
    for t in range(time):
        t+=1


        PreviousImage = CurrentImage
        CurrentImage = obs


        im = Image.fromarray(np.uint8(PreviousImage)).convert('RGB')
        im2 = Image.fromarray(np.uint8(CurrentImage)).convert('RGB')

        preprocess=transforms.Compose([
                transforms.Resize(64),
                transforms.ToTensor(), 
                ])

        im_preprocessed = preprocess(im2).to(device)
        previousim_preprocessed = preprocess(im).to(device)
        testCurrent = torch.unsqueeze(im_preprocessed, 0).to(device)
        testPrevious = torch.unsqueeze(previousim_preprocessed, 0).to(device)





        curr, _, _ = model.encode(testCurrent)
        prev, _, _ = model.encode(testPrevious)
        concatenatedTensor = torch.cat([prev, curr], dim=1)


        from IPython.display import clear_output
        out = model2(concatenatedTensor).cpu().detach().numpy() 
        #clear_output(wait=True)



        np.set_printoptions(precision=2)

        action = env.action_space.sample()
        action[0:3]=unnormalize(out)[0:3]
        
        obs, reward, done, _ = env.step(action)
        if render:
             env.render()
        r += reward
        
        
        
        if r> max_reward:
            max_reward = r
    #     print(type(action))

        #If we are doing this badly we are most likely off road. Let's cut compute time and cancel this simulation.
        if r <= -10:
            break
        
        scores.append(round(reward+0.1))
        
        if graph:
            if t%graphEveryX == 0:
                # get training result
                list1.append(r)

                # update learning curve
                line1.update(np.arange(len(list1)), list1)


    if graph:
        g.close()
    env.close()
    score_sum = r
    return score_sum

In [9]:
#Actor predicts action based off of state
#Actor network reuses most layers from NeuralNetwork - Find current conditions from image and freezes them during training
#Actor then has a few more layers that interpret these neurons, and try to maximise our critic network

#Critic analyses state and action and predicts our value function
#Critic uses "curiosity" as our value function
#What is curiosity?
#Curiosity explores states that are novel/unfamiliar. In order to achieve that, our agent will receive a high reward when exploring new trajectories.
#The network will have an intrinsic motivation to see new things (so it won't want to stay in the same spot)
#As such it will want to follow the road but it will also learn from mistakes when driving off road


In [90]:
ticks_start = 500
ticks_increment = 0
iterations_per_epoch = 2
num_epochs = 5

optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)  

model3.load_state_dict(torch.load('CurrentConditions.torch', map_location=device), strict=False)
model3.fc1.weight.requires_grad = False
model3.fc2.weight.requires_grad = False
model3.fc3.weight.requires_grad = False
model3.fc4.weight.requires_grad = False
model3.fc1.bias.requires_grad = False
model3.fc2.bias.requires_grad = False
model3.fc3.bias.requires_grad = False
model3.fc4.bias.requires_grad = False

for epoch in range(num_epochs):
    running_loss = 0
    simulation_ticks = ticks_start + epoch*ticks_increment
    for idx in range(iterations_per_epoch):
        
        score = runSimulator(simulation_ticks, render=False, graph=False, graphEveryX=10)
        running_loss += score
        score = torch.from_numpy(np.array([-score])).to(device)

        
        optimizer.zero_grad()
        #score.backward()
        optimizer.step()
        
        
        
    to_print = "Epoch[{}/{}] Score: {:.9f}".format(epoch+1,num_epochs, running_loss)#loss.data.item()/bs
    print(to_print)
    torch.save(model3.state_dict(), 'AdditivePolicy.torch')
    #print(epoch)
        
torch.save(model3.state_dict(), 'AdditivePolicy.torch')

#r = runSimulator(5000, render=False, graph=True, graphEveryX=10)
#print(r)


Track generation: 1060..1339 -> 279-tiles track
Track generation: 1239..1553 -> 314-tiles track
Epoch[1/5] Score: 45.331309904
Track generation: 1079..1353 -> 274-tiles track
Track generation: 1164..1459 -> 295-tiles track
Epoch[2/5] Score: 133.124018838
Track generation: 1138..1434 -> 296-tiles track
Track generation: 1240..1554 -> 314-tiles track
Epoch[3/5] Score: 45.421140413
Track generation: 1097..1383 -> 286-tiles track
Track generation: 1113..1395 -> 282-tiles track
Epoch[4/5] Score: 57.346494350
Track generation: 1160..1454 -> 294-tiles track
Track generation: 1099..1378 -> 279-tiles track
Epoch[5/5] Score: 131.333022319


In [91]:
r = runSimulator(2000, render=True, graph=True, graphEveryX=1)
#print(scores)

Track generation: 1096..1374 -> 278-tiles track
original backend: module://ipykernel.pylab.backend_inline
switch to backend: nbAgg


<IPython.core.display.Javascript object>

restore to backend: module://ipykernel.pylab.backend_inline
